<img style="float: left; padding-right: 10px; width: 45px" src="https://upload.wikimedia.org/wikipedia/fr/b/b1/Logo_EPF.png?raw=true"> 

**Introduction to Machine Learning - P2025: Energy & Environment**

## Lab 3B: Regularized linear models
*Credits*: Adapted from MDE's "Intro to ML" course by Y. Idrissi and I. Chafai.

**First name:**

**Last name:**

**Group:**


---

# Introduction
As we have seen during the course, **overfitting** is one of the most frequent issues we have to deal with while working on machine learning tasks. 

Overfitting refers to the scenario where a machine learning model can’t **generalize** or fit well on unseen data. It refers to a modeling error that occurs when a function corresponds too closely to a dataset. As a result, overfitting models may fail to fit additional data, and this may affect the accuracy of predicting future observations.

One way to avoid overfitting is to use **regularization**. It consists of adding a **penalty** term to the loss function we want to minimize. By doing this, we penalize the learning of complex models and encourage the learning of simpler models.

Another way of avoiding overfitting is to use proper **cross-validation** techniques in order to have an accurate estimation of the **generalization power** of our model (i.e how well it will perform on unseen data)

In this lab, we'll look at Ridge and Lasso regularization techniques for a logistic regression model. We will also see why it's important to use cross-validation when tuning the model's hyperparameters.

# Table of contents


# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')


In [ ]:
from scipy.stats import skew
from scipy.stats.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

In [ ]:
# setting up default plotting parameters
%matplotlib inline

plt.rcParams['figure.figsize'] = [20.0, 7.0]

sns.set_theme()
sns.set_context('notebook', font_scale=1.5)

# Dataset description
We will use [The Ames Housing dataset](http://jse.amstat.org/v19n3/decock.pdf) which is describing the sale of individual residential property in Ames, Iowa from 2006 to 2010. The data set contains 2930 observations and a large number of explanatory variables (23 nominal, 23 ordinal, 14 discrete, and 20 continuous) involved in assessing home values.

Take a look at the data_description.txt file to see a description of each of these explanatory variables.

In [ ]:
# read in data
data = pd.read_csv("../home/data/data.csv")

In [ ]:
# show the data size


118260

In [ ]:
# split to 80% train data and 20% test data


In [ ]:
# have a look at the data


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
254,255,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1957,1957,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,CBlock,TA,TA,No,Rec,922,Unf,0,392,1314,GasA,...,Y,SBrkr,1314,0,0,1314,1,0,1,0,3,1,TA,5,Typ,0,NaN,Attchd,1957.0,RFn,1,294,TA,TA,Y,250,0,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,145000
1066,1067,60,RL,59.0,7837,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,7,1993,1994,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,799,799,GasA,...,Y,SBrkr,799,772,0,1571,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1993.0,RFn,2,380,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal,178000
638,639,30,RL,67.0,8777,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Feedr,Norm,1Fam,1Story,5,7,1910,1950,Gable,CompShg,MetalSd,Wd Sdng,None,0.0,TA,TA,CBlock,Fa,TA,No,Unf,0,Unf,0,796,796,GasA,...,Y,FuseA,796,0,0,796,0,0,1,0,2,1,TA,4,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,P,328,0,164,0,0,0,NaN,MnPrv,NaN,0,5,2008,WD,Normal,85000
799,800,50,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,SWISU,Feedr,Norm,1Fam,1.5Fin,5,7,1937,1950,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,252.0,TA,TA,BrkTil,Gd,TA,No,ALQ,569,Unf,0,162,731,GasA,...,Y,SBrkr,981,787,0,1768,1,0,1,1,3,1,Gd,7,Typ,2,TA,Detchd,1939.0,Unf,1,240,TA,TA,Y,0,0,264,0,0,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal,175000
380,381,50,RL,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,SWISU,Norm,Norm,1Fam,1.5Fin,5,6,1924,1950,Gable,CompShg,BrkFace,Wd Sdng,None,0.0,TA,TA,BrkTil,TA,TA,No,LwQ,218,Unf,0,808,1026,GasA,...,Y,SBrkr,1026,665,0,1691,0,0,2,0,3,1,Gd,6,Typ,1,Gd,Detchd,1924.0,Unf,1,308,TA,TA,Y,0,0,242,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,127000


## Data preprocessing: 

**Exercise :** Plot the distribution of the price variable

In [ ]:
matplotlib.rcParams['figure.figsize'] = (12.0, 6.0)
prices = pd.DataFrame({"price":train["SalePrice"]})
prices.hist()

We can see that the distribution is highly skewed to the left. As we have seen in the feature engineering class, one way to deal with the skewness is to apply a log transformation to be as close to a normal distribution as possible.

**Exercise :** Plot the distribution of the log of the price variable, you can use the log1p numpy function.

In [ ]:
# concatenate train and test data before transformation
all_data = pd.concat((train.loc[:,'MSSubClass':'SaleCondition'],
                      test.loc[:,'MSSubClass':'SaleCondition']))

In [ ]:
#log transform the target:
train["SalePrice"] = np.log1p(train["SalePrice"])
test["SalePrice"] = np.log1p(test["SalePrice"])

#log transform skewed numeric features (skewness > 0.75):
numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
skewed_feats = skewed_feats[skewed_feats > 0.75]
skewed_feats = skewed_feats.index

all_data[skewed_feats] = np.log1p(all_data[skewed_feats])

In [ ]:
## Create dummy variables


In [ ]:
#filling numerical columns NA's with the mean of the columns on the train dataset:


In [ ]:
#create matrices for sklearn:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y_train = train.SalePrice
y_test = test.SalePrice

# Models


Ridge model is a regression model where the loss function is the linear least squares function and regularization is given by the l2-norm.
The objective function is given by: 
$$ {||y - Xw||^2_2 + alpha * ||w||^2_2 }$$

The main tuning parameter for the Ridge model is alpha - a regularization parameter that measures how flexible our model is. The higher the regularization the less prone our model will be to overfit. However it will also lose flexibility and might not capture all of the signal in the data.

In [ ]:
model_ridge = Ridge()

**Exercise**: Perform a grid search in which you try different values of alpha and where the goal is to have the lowest RMSE on the train dataset.

The values to use for alpha are: 
0, 0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75


In [ ]:
## Function to compute the RMSE on the train dataset
def compute_train_rmse(model):
    model.fit(X_train,y_train)
    rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
    return(rmse)

**Exercise**: Plot the train RMSE for the different values of alpha.

Which value gives the best (lowest) RMSE ?



Which alpha yields the best model ?

Now let's see how this holds on the test data.

**Exercise**: Instead of evaluating on the train data, now evaluate the RMSE on the test data.

In [ ]:
## Function to compute the RMSE on the test dataset
def compute_test_rmse(model):
    model.fit(X_train,y_train)
    rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
    return(rmse)

**Exercise**: Plot the test RMSE for the different values of alpha.

How does the best performing alpha on training test perform on the test set? What happened?

**Importance of cross-validation**

Let's now perform the grid search while using cross-validation to evaluate our models. For this, you can use the cross_val_score function that is provided by scikit-learn.

In [ ]:
def rmse_cv(model):
    rmse= np.sqrt(-cross_val_score(model, X_train, y_train, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)

The main tuning parameter for the Ridge model is alpha - a regularization parameter that measures how flexible our model is. The higher the regularization the less prone our model will be to overfit. However it will also lose flexibility and might not capture all of the signal in the data.

In [ ]:
##Plot the cross-validation result without the first alpha value (0)


Note the U-ish shaped curve above. When alpha is too large the regularization is too strong and the model cannot capture all the complexities in the data. If however we let the model be too flexible (alpha small) the model begins to overfit. A value of alpha = 10 is about right based on the plot above.

Let' try out the Lasso model. We will do a slightly different approach here and use the built in Lasso CV to figure out the best alpha for us. 


The lasso performs even better so we'll just use this one to predict on the test set. Another good thing about the Lasso is that it does feature selection for you - setting coefficients of features it deems unimportant to zero. Let's take a look at the coefficients:

We can also take a look directly at what the most important coefficients are: